In [3]:
import timeit
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json

from numpy import expand_dims
from pytorch_lightning import Trainer
from matplotlib import pyplot
from PIL import Image
from torch.utils.data import DataLoader

from keras.preprocessing.image import ImageDataGenerator  
from keras.utils import load_img, img_to_array
from anomalib.models import get_model
from anomalib.models.padim.lightning_model import Padim
from anomalib.config import get_configurable_parameters
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks
from anomalib.data.folder import Folder

c:\Users\Pablo\anaconda3\envs\anomalib_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def img_resize(path, foldertype, newfolder):
    imgs = os.listdir(path)
    for img in imgs:
        img_arr = np.array(Image.open(path+'/'+img)) # (288, 352, 3)  
        img_new = cv2.resize(img_arr, (288,288))
        im = Image.fromarray(img_new)
        im.save(f'./data/{newfolder}/{foldertype}/{img}')

def resize_img_pipe(oldfolder, newfolder):
    os.mkdir(f'./data/{newfolder}')
    os.mkdir(f'./data/{newfolder}/abnormal')
    os.mkdir(f'./data/{newfolder}/normal')

    img_resize(f'./data/{oldfolder}/normal/','normal',newfolder)
    img_resize(f'./data/{oldfolder}/abnormal/','abnormal', newfolder)

def image_augmenter(path, folder, n, newfolder):
    img = img_to_array(load_img(path))
    data = expand_dims(img, 0)
    datagen = ImageDataGenerator(#rotation_range=40,
                                    width_shift_range=0.2, 
                                    #featurewise_center=True,
                                    #featurewise_std_normalization=True,
                                    height_shift_range=0.2, 
                                    #shear_range=0.2, 
                                    #zoom_range=0.2,
                                    #rotation_range=90,
                                    brightness_range=[0.4,1.3],
                                    horizontal_flip=True,
                                    #zca_whitening=True,
                                    vertical_flip=True,
                                    fill_mode='nearest')
    count = 0                                
    for batch in datagen.flow(data, batch_size=1, save_prefix='orange', save_to_dir=f'./data/{newfolder}/{folder}',save_format='jpg'):
        count += 1
        if count == n:
            break
     
def gen_img_pipe(oldfolder, newfolder, n, generate):
    resize_img_pipe(oldfolder, newfolder)
    if generate:
        normal = os.listdir(f'./data/{newfolder}/normal')
        abnormal =  os.listdir(f'./data/{newfolder}/abnormal')
        for end in normal:
            image_augmenter(f'./data/{newfolder}/normal/{end}','normal', n, newfolder)
        for end in abnormal:
            image_augmenter(f'./data/{newfolder}/abnormal/{end}', 'abnormal', n, newfolder)

In [12]:
gen_img_pipe('naranjas_pynq_0','naranjas_pynq_2_crop_augmentation', 10, True)

In [13]:
def anomaset(model):
    CONFIG_PATH = f"./anomalib/models/{model}/custom.yaml"
    # pass the config file to model, callbacks and datamodule
    config = get_configurable_parameters(config_path=CONFIG_PATH)
    
    #config["dataset"]["name"] = "small_18" 
    #config["model"]["backbone"] = "resnet18"  #resnet18, wide_resnet50_2, cait_m48_448, deit_base_distilled_patch16_384
    datamodule = Folder(
        root="./data/naranjas_pynq_2_crop_augmentation",
        image_size=288,
        #task='classification',
        seed=42)

    datamodule.setup()
    #model = Padim(input_size=[288, 288], backbone="wide_resnet50_2", layers=['layer1','layer2','layer3'],pre_trained=True)
    model = get_model(config) # (288, 352)
    callbacks = get_callbacks(config)

    return datamodule, model, callbacks, config

In [7]:
def anomatrain(datamodule, model, callbacks, config):
    # start training
    trainer = Trainer(**config.trainer, callbacks=callbacks)
    trainer.fit(model=model, datamodule=datamodule)
    # load best model from checkpoint before evaluating
    load_model_callback = LoadModelCallback(weights_path=trainer.checkpoint_callback.best_model_path)
    trainer.callbacks.insert(0, load_model_callback)
    return trainer

In [8]:
def anomatest(trainer, model, datamodule):
    trainer.test(model=model, datamodule=datamodule)
    pixAUC = round(float(trainer.logged_metrics['image_F1Score']),4)
    imgAUC = round(float(trainer.logged_metrics['image_AUROC']),4)

    return pixAUC, imgAUC

In [9]:
def anomastudy(model):
    datamodule, model, callbacks, config = anomaset(model)

    trainer = anomatrain(datamodule, model, callbacks, config)
    
    pixAUC, imgAUC = anomatest(trainer, model, datamodule)

    return pixAUC, imgAUC

In [10]:
def anomaresults(models):
    pix, img = {}, {}

    for model in models:

        pixAUC, imgAUC = anomastudy(model)

        pix[model], img[model] = pixAUC, imgAUC

    data_pix = pd.DataFrame.from_dict(pix, orient='index')
    data_img = pd.DataFrame.from_dict(img, orient='index')
    
    return data_pix, data_img

In [14]:
models = ['patchcore']#,'draem','dfm', 'cflow', 'stfpm', 'ganomaly', 'dfkde', 'patchcore']

data_f1, data_img = anomaresults(models)

C:\Users\Pablo\anomalib\anomalib\config\config.py:147: UserWarning: The seed value is now fixed to 0. Up to v0.3.7, the seed was not fixed when the seed value was set to 0. If you want to use the random seed, please select `None` for the seed value (`null` in the YAML file) or remove the `seed` key from the YAML file.
  warn(
C:\Users\Pablo\anomalib\anomalib\config\config.py:175: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(
Transform configs has not been provided. Images will be normalized using ImageNet statistics.
Transform configs has not been provided. Images will be normalized using ImageNet statistics.
c:\Users\Pablo\anaconda3\envs\anomalib_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warning

Epoch 0:  64%|██████▍   | 9/14 [01:40<00:55, 11.18s/it, loss=nan, v_num=1]

c:\Users\Pablo\anaconda3\envs\anomalib_env\lib\site-packages\pytorch_lightning\trainer\trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
#model = torchvision.models.resnet101()
#for name, layer in model.named_modules():
#    print(name)